<a href="https://colab.research.google.com/github/kushb2/Techno-Funda/blob/main/Snippets_Importing_libraries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# ==============================================================================
# PROJECT: INSTITUTIONAL SNIPER (v6.1 - ROBUST DATA EDITION)
# OBJECTIVE: Institutional Footprint Detection with Interactive History
# ==============================================================================

import yfinance as yf
import pandas as pd
import numpy as np
import time
import requests
from datetime import datetime, timedelta
from google.colab import data_table

# Enable interactive tables for all dataframes
data_table.enable_dataframe_formatter()

# Try importing nsepython for the specific latest-data check
try:
    from nsepython import nse_quote
except ImportError:
    nse_quote = None
    print("⚠️ NSEPython not found. Please install: pip install nsepython")

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
class Config:
    # --- YOUR WATCHLIST ---
    WATCHLIST = [
        "NETWEB.NS"
    ]

    # --- The Sniper Scope ---
    VOL_LOOKBACK = 20
    Z_SCORE_LIMIT = 2.5       # Slightly tighter for "Active" tag
    DELIVERY_LIMIT = 0.45     # 45% Delivery

    # --- Trend Definitions ---
    SMA_SHORT_TERM = 20       # Short term trend
    SMA_LONG_TERM = 200       # Major trend

# ==============================================================================
# 2. DATA ENGINE (ROBUST FETCHING)
# ==============================================================================
def fetch_historical_delivery_robust(symbol, days=30):
    """
    CORRECT WAY: Manually fetches historical data from NSE by mimicking a real
    browser to bypass the 'Access Denied' blocks on Colab.
    """
    try:
        # 1. Setup Headers to look like a real Chrome Browser
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive',
            'Referer': 'https://www.nseindia.com/'
        }

        # 2. Initialize Session & Get Cookies (Crucial Step)
        session = requests.Session()
        session.headers.update(headers)
        # Visit homepage first to set cookies
        session.get("https://www.nseindia.com", timeout=10)

        # 3. Construct URL
        clean_sym = symbol.replace(".NS", "").replace("&", "%26")
        end_date = datetime.now().strftime("%d-%m-%Y")
        start_date = (datetime.now() - timedelta(days=days)).strftime("%d-%m-%Y")

        # NSE API Endpoint for Historical Data
        url = f"https://www.nseindia.com/api/historical/cm/equity?symbol={clean_sym}&series=[%22EQ%22]&from={start_date}&to={end_date}"

        # 4. Fetch Data
        response = session.get(url, timeout=10)

        if response.status_code == 200:
            json_data = response.json()
            data_list = json_data.get('data', [])

            if not data_list: return pd.DataFrame()

            # 5. Parse into DataFrame
            df = pd.DataFrame(data_list)

            # Standardize Date
            df['CH_TIMESTAMP'] = pd.to_datetime(df['CH_TIMESTAMP'])
            df.set_index('CH_TIMESTAMP', inplace=True)

            # Extract Delivery %
            # NSE key is usually 'COP_DELIV_PERC' (0.45 or 45.0)
            if 'COP_DELIV_PERC' in df.columns:
                df['Delivery_Pct'] = df['COP_DELIV_PERC'].astype(float)
            else:
                df['Delivery_Pct'] = 0.0

            return df[['Delivery_Pct']]
        else:
            # print(f"⚠️ Blocked by NSE (Status: {response.status_code})")
            return pd.DataFrame()

    except Exception as e:
        return pd.DataFrame()

def fetch_data(ticker):
    try:
        stock = yf.Ticker(ticker)

        # 1. Get OHLCV History (Yahoo)
        hist = stock.history(period="3mo")
        if hist.empty: return None

        # 2. Get Delivery History (NSE Robust)
        delivery_df = fetch_historical_delivery_robust(ticker)

        # 3. Merge Data
        # Normalize dates to remove time components for clean merging
        hist.index = pd.to_datetime(hist.index).normalize()

        if not delivery_df.empty:
            delivery_df.index = pd.to_datetime(delivery_df.index).normalize()
            # Join: Keep Yahoo rows, add NSE delivery where dates match
            hist = hist.join(delivery_df, how='left')
            hist['Delivery_Pct'] = hist['Delivery_Pct'].fillna(0)
        else:
            hist['Delivery_Pct'] = 0.0

        return hist
    except:
        return None

# ==============================================================================
# 3. LOGIC LAYER (The Brains)
# ==============================================================================
def apply_sniper_logic(df):
    """Adds Technical Indicators and Smart Money Flags."""

    # 1. Trend Indicators
    df['SMA_20'] = df['Close'].rolling(window=Config.SMA_SHORT_TERM).mean()
    df['Trend'] = np.where(df['Close'] > df['SMA_20'], '🟢 Up', '🔴 Down')

    # 2. Volume Anomalies
    df['Vol_Mean'] = df['Volume'].rolling(window=Config.VOL_LOOKBACK).mean()
    df['Vol_Std'] = df['Volume'].rolling(window=Config.VOL_LOOKBACK).std()

    # Z-Score Calculation
    # Avoid division by zero
    df['Vol_Z_Score'] = np.where(df['Vol_Std'] == 0, 0, (df['Volume'] - df['Vol_Mean']) / df['Vol_Std'])

    # 3. Institutional Activity Flag
    # Note: NSE typically returns 45.5 for 45.5%, but sometimes 0.45.
    # Logic: If max delivery in column > 1, assume scale is 0-100. Else scale is 0-1.
    scale_factor = 1.0
    if df['Delivery_Pct'].max() > 1.0:
        scale_factor = 100.0 # Data is already 45.0
    else:
        scale_factor = 1.0   # Data is 0.45

    threshold = Config.DELIVERY_LIMIT * scale_factor

    conditions = (
        (df['Vol_Z_Score'] > Config.Z_SCORE_LIMIT) &
        (df['Delivery_Pct'] > threshold)
    )

    df['Smart_Money_Active'] = np.where(conditions, '🔥🔥 YES', '-')

    return df

# ==============================================================================
# 4. REPORTING
# ==============================================================================
def run_forensic_analysis():
    print(f"🕵️  STARTING INSTITUTIONAL FORENSIC SCAN...")
    print(f"   (Fetching 20-Day History with Robust Delivery Check)\n")

    for ticker in Config.WATCHLIST:
        print(f"Analyzing: {ticker} ...")

        # 1. Fetch & Process
        df = fetch_data(ticker)
        if df is None:
            print(f"Skipping {ticker} (No Data)")
            continue

        df = apply_sniper_logic(df)

        # 2. Filter for Display (Last 20 Days only)
        display_cols = [
            'Close', 'Trend', 'Volume', 'Vol_Z_Score', 'Delivery_Pct', 'Smart_Money_Active'
        ]

        last_20_days = df.tail(20)[display_cols].copy()

        # Formatting
        last_20_days['Close'] = last_20_days['Close'].round(2)
        last_20_days['Vol_Z_Score'] = last_20_days['Vol_Z_Score'].round(1)

        # Handle Delivery Formatting gracefully
        # If it's > 1 (e.g. 45.0), add %. If < 1 (e.g. 0.45), multiply by 100 then add %
        if last_20_days['Delivery_Pct'].max() <= 1.0 and last_20_days['Delivery_Pct'].max() > 0:
             last_20_days['Delivery_Pct'] = (last_20_days['Delivery_Pct'] * 100).round(1).astype(str) + '%'
        else:
             last_20_days['Delivery_Pct'] = last_20_days['Delivery_Pct'].round(1).astype(str) + '%'

        last_20_days['Volume'] = (last_20_days['Volume'] / 100000).round(2).astype(str) + ' L'

        # Sort Newest First
        last_20_days = last_20_days.sort_index(ascending=False)

        # 3. RENDER TABLE
        print(f"📊 DEEP DIVE: {ticker}")
        display(data_table.DataTable(last_20_days, num_rows_per_page=10))

        # Summary Alert
        if '🔥🔥 YES' in last_20_days['Smart_Money_Active'].head(1).values:
             print(f"🚨 ALERT: INSTITUTIONAL BUYING DETECTED TODAY in {ticker}!")
        print("-" * 60)

if __name__ == "__main__":
    run_forensic_analysis()

🕵️  STARTING INSTITUTIONAL FORENSIC SCAN...
   (Fetching 20-Day History with Robust Delivery Check)

Analyzing: NETWEB.NS ...
📊 DEEP DIVE: NETWEB.NS


,Close,Trend,Volume,Vol_Z_Score,Delivery_Pct,Smart_Money_Active
Date,,,,,,
2026-01-14 00:00:00+05:30,3187.6,🔴 Down,7.35 L,-0.2,0.0%,-
2026-01-13 00:00:00+05:30,3273.0,🟢 Up,7.75 L,-0.2,0.0%,-
2026-01-12 00:00:00+05:30,3346.4,🟢 Up,14.9 L,0.3,0.0%,-
2026-01-09 00:00:00+05:30,3310.1,🟢 Up,14.39 L,0.3,0.0%,-
2026-01-08 00:00:00+05:30,3397.4,🟢 Up,15.88 L,0.4,0.0%,-
2026-01-07 00:00:00+05:30,3368.4,🟢 Up,15.73 L,0.4,0.0%,-
2026-01-06 00:00:00+05:30,3348.1,🟢 Up,30.77 L,1.4,0.0%,-
2026-01-05 00:00:00+05:30,3274.6,🟢 Up,71.64 L,4.2,0.0%,-
2026-01-02 00:00:00+05:30,3024.1,🔴 Down,2.94 L,-0.4,0.0%,-


------------------------------------------------------------


In [26]:
from nsepython import nse_quote

def get_real_delivery(ticker):
    # Standardize symbol (remove .NS if present)
    sym = ticker.replace(".NS", "")

    print(f"Fetching trade info for {sym}...")

    # We use the 'trade_info' endpoint specifically
    try:
        # This function fetches specific trade details including delivery
        trade_info = nse_quote(sym, section="trade_info")

        # Check if we got valid data back
        if trade_info and 'securityWiseDP' in trade_info:
            delivery_data = trade_info['securityWiseDP']
            print(f"   ✅ Raw Data Found: {delivery_data}")

            # Extract the actual percentage
            delivery_pct = float(delivery_data.get('deliveryToTradedQuantity', 0))
            return delivery_pct

        else:
            print(f"   ⚠️ No delivery data found in response.")
            return 0.0

    except Exception as e:
        print(f"   ❌ Error: {e}")
        return 0.0

# Test it
pct = get_real_delivery("NETWEB")
print(f"\n🎯 Final Delivery %: {pct}%")

Fetching trade info for NETWEB...
   ✅ Raw Data Found: {'quantityTraded': 735503, 'deliveryQuantity': 151124, 'deliveryToTradedQuantity': 20.55, 'seriesRemarks': None, 'secWiseDelPosDate': '14-JAN-2026 EOD'}

🎯 Final Delivery %: 20.55%


In [1]:
# ==============================================================================
# PROJECT: TECHNO-FUNDA INSTITUTIONAL SCANNER (v2.0)
# OBJECTIVE: Identify Safe Stocks with Institutional Buying Momentum
# AUTHOR: You & Gemini
# ==============================================================================

# ------------------------------------------------------------------------------
# TABLE OF CONTENTS
# ------------------------------------------------------------------------------
# 1. IMPORTS & SETUP ......... Library imports and environment checks
# 2. CONFIGURATION ........... Central control panel for all thresholds
# 3. DATA ENGINE ............. Handles yfinance and NSE connections
# 4. STRATEGY LAYERS ......... The 3-Step Filtering Logic
# 5. MAIN CONTROLLER ......... The execution loop and reporting
# ------------------------------------------------------------------------------
#




fatal: not a git repository (or any of the parent directories): .git


In [28]:
import pandas as pd
from datetime import datetime, timedelta

try:
    from nsepython import equity_history
except ImportError:
    print("⚠️ NSEPython not found. Please install with: pip install nsepython")
    raise

def debug_historical_delivery(symbol="RELIANCE"):
    print(f"🕵️ DEBUGGING HISTORICAL DATA FOR: {symbol}")

    end_date = datetime.now().strftime("%d-%m-%Y")
    start_date = (datetime.now() - timedelta(days=10)).strftime("%d-%m-%Y")
    print(f"📅 Fetching from {start_date} to {end_date}...")

    try:
        df = equity_history(symbol, "EQ", start_date, end_date)

        if df is None:
            print("❌ RESULT: equity_history() returned None. Function likely broken for current NSE site.")
            return

        if df.empty:
            print("❌ RESULT: Empty DataFrame. Check if NSE blocked the request or the function is deprecated.")
            return

        print("✅ RESULT: Data Received!")
        print(f"   Rows Fetched: {len(df)}")
        print(f"   Columns Found: {df.columns.tolist()}")

        delivery_col = None
        for col in [
            "COP_DELIV_PERC",
            "deliveryToTradedQuantity",
            "% Dly Qt to Traded Qty",
            "%DlyQttoTradedQty",      # common in other NSE libs
            "DeliverableQty"
        ]:
            if col in df.columns:
                delivery_col = col
                break

        if delivery_col:
            ts_col = "CH_TIMESTAMP" if "CH_TIMESTAMP" in df.columns else "Date"
            cols_to_show = [c for c in [ts_col, delivery_col] if c in df.columns]
            print(f"\n🎯 FOUND DELIVERY COLUMN: '{delivery_col}'")
            print(df[cols_to_show].head())
        else:
            print("\n⚠️ WARNING: Data received, but NO Delivery column found.")
            print("First row dump:")
            print(df.iloc[0])

    except Exception as e:
        print(f"❌ CRITICAL ERROR: {type(e).__name__}: {e}")

# Run the test
debug_historical_delivery("SBIN")


🕵️ DEBUGGING HISTORICAL DATA FOR: SBIN
📅 Fetching from 04-01-2026 to 14-01-2026...
❌ CRITICAL ERROR: KeyError: 'data'


In [46]:
# ==============================================================================
# PROJECT: INSTITUTIONAL SNIPER (v17 - DYNAMIC DELIVERY & DATE FIX)
# LOGIC: Median-based Vol Z-Score + Delivery Z-Score (Deviation from Mean)
# ==============================================================================

import yfinance as yf
import pandas as pd
import numpy as np
import requests
import io
import time
from datetime import datetime, timedelta
from google.colab import data_table

data_table.enable_dataframe_formatter()

try:
    from nsepython import nse_quote
except ImportError:
    print("⚠️ NSEPython not found. Please run: !pip install nsepython")

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
class Config:
    WATCHLIST = [ "NETWEB.NS"]

    # --- Dynamic Thresholds ---
    VOL_Z_LIMIT = 2.0      # How many std devs volume is above typical
    DELIV_Z_LIMIT = 1.5    # How many std devs delivery is above typical

    HISTORY_LOOKBACK = 40  # Days to build history baseline

# ==============================================================================
# 2. DATA ENGINE (Archive + Snapshot)
# ==============================================================================
def build_delivery_database(watchlist, days_back=40):
    clean_watchlist = [x.replace(".NS", "") for x in watchlist]
    database = {sym: {} for sym in clean_watchlist}
    headers = {'User-Agent': 'Mozilla/5.0'}

    for i in range(1, days_back + 1):
        date_obj = datetime.now() - timedelta(days=i)
        if date_obj.weekday() > 4: continue

        date_str = date_obj.strftime("%d%m%Y")
        formatted_date = date_obj.strftime("%Y-%m-%d")
        url = f"https://nsearchives.nseindia.com/products/content/sec_bhavdata_full_{date_str}.csv"

        try:
            with requests.Session() as s:
                r = s.get(url, headers=headers, timeout=2)
                if r.status_code == 200:
                    csv_data = pd.read_csv(io.StringIO(r.text))
                    csv_data.columns = [c.strip() for c in csv_data.columns]
                    mask = csv_data['SYMBOL'].isin(clean_watchlist)
                    for _, row in csv_data[mask].iterrows():
                        val = row['DELIV_PER']
                        if val != '-' and pd.notna(val):
                            database[row['SYMBOL']][formatted_date] = float(val)
        except: pass
    return database

# ==============================================================================
# 3. CORE SNIPER LOGIC
# ==============================================================================
def analyze_stock(ticker, delivery_db):
    clean_sym = ticker.replace(".NS", "")
    stock = yf.Ticker(ticker)
    df = stock.history(period="6mo")
    if df.empty: return None

    # --- Date & Holiday Cleaning ---
    df.index = df.index.tz_localize(None).normalize()
    df = df[df['Volume'] > 0].copy()

    # --- 1. Dynamic Volume Analysis (Robust Median) ---
    df['Vol_Median'] = df['Volume'].rolling(window=20).median()
    df['Vol_Std'] = df['Volume'].rolling(window=20).std()
    df['Vol_Z'] = (df['Volume'] - df['Vol_Median']) / df['Vol_Std']

    # --- 2. Dynamic Delivery Analysis ---
    # Convert DB dictionary to Series and merge
    if clean_sym in delivery_db:
        del_series = pd.Series(delivery_db[clean_sym])
        del_series.index = pd.to_datetime(del_series.index)
        df = df.join(del_series.rename('Deliv_Val'), how='left')
    else:
        df['Deliv_Val'] = np.nan

    # Inject Today's Snapshot into Deliv_Val
    try:
        trade_info = nse_quote(clean_sym, section="trade_info")
        today_val = float(trade_info['securityWiseDP'].get('deliveryToTradedQuantity', 0))
        df.iloc[-1, df.columns.get_loc('Deliv_Val')] = today_val
    except: pass

    # Calculate Delivery Deviation
    df['Deliv_Mean'] = df['Deliv_Val'].rolling(window=20).mean()
    df['Deliv_Std'] = df['Deliv_Val'].rolling(window=20).std()
    # Deliv_Z measures: "Is today's delivery unusual for THIS stock?"
    df['Deliv_Z'] = (df['Deliv_Val'] - df['Deliv_Mean']) / df['Deliv_Std']

    # --- 3. Trend & Formatting ---
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['Trend'] = np.where(df['Close'] > df['SMA_20'], '🟢 Up', '🔴 Down')

    # Final Table Prep
    view_df = df.tail(20).copy()
    view_df = view_df.sort_index(ascending=False)

    report = []
    for idx, row in view_df.iterrows():
        # Signal Logic
        action = "-"
        v_z = row['Vol_Z']
        d_z = row['Deliv_Z']

        if v_z > Config.VOL_Z_LIMIT and d_z > Config.DELIV_Z_LIMIT:
            action = "🔥🔥 BUY"
        elif v_z > Config.VOL_Z_LIMIT:
            action = "⚠️ High Vol"
        elif d_z > 2.0:
            action = "👀 Hidden Accum"

        report.append({
            "Date": idx.strftime('%Y-%m-%d'),
            "Close": round(row['Close'], 2),
            "Trend": row['Trend'],
            "Volume": f"{round(row['Volume']/100000, 2)} L",
            "Vol Z": round(v_z, 1),
            "Deliv %": f"{row['Deliv_Val']}%" if pd.notna(row['Deliv_Val']) else "-",
            "Deliv Z": round(d_z, 1) if pd.notna(d_z) else 0,
            "Action": action
        })

    return pd.DataFrame(report)

# ==============================================================================
# 4. EXECUTION
# ==============================================================================
def run_scanner():
    db = build_delivery_database(Config.WATCHLIST, days_back=Config.HISTORY_LOOKBACK)
    print(f"\n🕵️  DYNAMIC SCANNER ACTIVE")
    print(f"Flags: Vol_Z > {Config.VOL_Z_LIMIT} & Deliv_Z > {Config.DELIV_Z_LIMIT}\n")

    for ticker in Config.WATCHLIST:
        print(f"Scanning {ticker}...", end="\r")
        result = analyze_stock(ticker, db)
        if result is not None:
            print(f"📊 {ticker}" + " " * 30)
            display(data_table.DataTable(result, num_rows_per_page=10, include_index=False))
            print("-" * 60)

if __name__ == "__main__":
    run_scanner()


# Understaning notes




1. Debt-to-Equity < 1.5 (The "Mortgage" Analogy)
Think of this like buying a house.

Equity is your Down Payment (the money you actually own in the house).

Debt is the Mortgage (the money you borrowed from the bank).

If you put ₹10 Lakhs down and borrow ₹90 Lakhs, your ratio is 9.0. You are "highly leveraged." If the house price drops even a little, you are in trouble. If you put ₹50 Lakhs down and borrow ₹50 Lakhs, your ratio is 1.0. You are "safe."

The Rule (< 1.5): For every ₹1 of the shareholders' own money, the company should not owe more than ₹1.50 to the bank.

Why? In bad times (like a recession), companies with high debt go bankrupt because they must pay interest regardless of whether they made a profit. Low debt companies can survive the storm.

2. ROE > 10% (The "Savings Account" Analogy)
ROE (Return on Equity) asks: "For every ₹100 of shareholder money, how much profit did the management generate this year?"

Your Question: "I think this should be higher?"

The Answer: You are absolutely right! You are thinking like a "Growth Investor." Most top-tier investors look for 15% or 20%+.

Why use 10% then? 10% is just the "Safety Floor." It filters out the "garbage" (companies losing money or barely breaking even).

A savings account gives you ~7% risk-free.

If a risky company can't even generate 10%, it's not worth your time.

1. VOL_LOOKBACK = 20 (The Baseline)
This tells the computer how far back to look to define what is "normal" for this stock.

Layman Term: "The Monthly Average."

Why 20? There are roughly 20 trading days in a month.

Example: Imagine a coffee shop ☕. To know if today is unusually busy, you can't just look at yesterday (which might have been a holiday). You look at the average sales over the last 20 days to get a solid baseline of what "normal" looks like.

2. Z_SCORE_LIMIT = 2.0 (The Anomaly Detector)
This measures how "weird" or "extreme" today's volume is compared to that baseline.

Layman Term: "The Excitement Level."

The Math: A Z-Score of 0 is perfectly normal. A Z-Score of +2.0 means volume is significantly higher than usual (statistically rare).

Example:

Normal day: The coffee shop sells 100 cups.

Busy day (Z=1): They sell 120 cups. (Busy, but happens).

Crazy day (Z=2): They sell 160 cups. Something special is definitely happening! 😲

3. DELIVERY_LIMIT = 40.0 (The Commitment Check)
This checks what percentage of the shares traded were actually "taken home" (delivered to a demat account) versus just bought and sold rapidly (intraday trading).

Layman Term: "The Serious Buyers."

Why 40%? If people are just gambling for quick profits, they sell before the market closes (Delivery = 0%). If they are investing for the long term, they keep the shares (Delivery = 100%).

Example:

Low Delivery (20%): A crowd enters the coffee shop, looks around, takes selfies, and leaves without buying much. Lots of noise, no real business.

High Delivery (40%+): The crowd enters, and almost half of them sit down and order a full meal. They are committed customers. ✅